# The script to retrieve Auth_Id's

Note the "apply to dataframe" example. I use the intution here to apply this to the full dataset

In [ ]:
# Some imports
import pandas as pd


frame = pd.read_csv('Vtech.csv')
frame

In [ ]:
# We're going to Split fullname to two cols to get first and last name

names = {0:'first',1:'last'}

df = frame['Fullname'].str.split(' ',expand=True).rename(columns = names).fillna('')
df = df.drop(columns = [2,3], axis =1)

In [ ]:
frame['first']=df['first']
frame['last'] = df['last']

frame = frame[ ['last'] + [ col for col in frame.columns if col != 'last' ] ]
frame = frame[ ['first'] + [ col for col in frame.columns if col != 'first' ] ]
frame

#frame['first','last'] = df


In [ ]:
## Define a function to apply api calls over dataframe

from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import json
    
## Load configuration (contains my API key for scopus)
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

## Initialize client
client = ElsClient(config['apikey'])


#Based almost entirely on function from CV_Prog_beta
#Try, Except passes over errors with api calls

def auth_looker(first,last):
    auth_srch = ElsSearch(('authlast(%s) and authfirst(%s)and affil(Virginia Polytechnic Institute)' %(last,first)),'author')
    auth_srch.execute(client)
    #Can change Affil value to increase coverage (Some auths use different institution names) 
    
    try:
        
        ids = [(item['dc:identifier']) for item in auth_srch.results]
    
    except:
        pass
    
    return(ids)

# This is the function that allows us to use elements of our dataframe as arguments in the
# auth_looker function. Function then "enriches" original dataframe with new 'id' column


def enrich_w_auth(row):
    column_first = 'first'
    column_last = 'last'
    first_val = row[column_first]
    second_val = row[column_last]
    try:
        
        auth_ids = (auth_looker(first_val,second_val))
        row['id'] = auth_ids
    
    except:
        row['id'] = ''
    return row

In [12]:
# Run (takes 5 seconds/scrape)
    
df = frame.apply(enrich_w_auth, axis = 1)

In [26]:
#

df_copy = df.copy()
df_copy[['Fullname','id']].describe()


,Fullname,id
count,2599,2600
unique,2599,2158
top,Saifur Rahman,
freq,1,440


In [ ]:
#some cleaning (my api calls weren't perfect)

df_copy = df.copy()
df_copy['id'] = [(i[0].split(':')) for i in df_copy['id']]
df_copy['id'] = [int(i[-1]) for i in df_copy['id']]
df_copy

In [28]:
df.to_csv('AUTH_ID_DATA.csv')